# Импорт библиотек и чтение файлов

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 
import numpy as np
import string
from tqdm import tqdm_notebook as tqdm
import re

#NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

In [2]:
df_movies = pd.read_csv(r'movies_metadata.csv')
df_movies.shape

C:\Users\isaev\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(45466, 24)

# Preprocessing

*Проверим на наличие NaN*

In [3]:
df_movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [4]:
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


*Удалим столбцы с большим количеством NaN и столбцы, которые нам не особо нужны, чтоб уменьшим размер датафрейма*

In [5]:
df_movies['original_title'] = df_movies['title']

In [6]:
df_movies.drop(df_movies.columns.difference(['original_title', 'overview', 'id']), 1, inplace=True)

C:\Users\isaev\AppData\Local\Temp/ipykernel_3780/868513532.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_movies.drop(df_movies.columns.difference(['original_title', 'overview', 'id']), 1, inplace=True)


In [7]:
id_film = df_movies['id'].nunique()

In [8]:
df_movies = df_movies.fillna(0)
df_movies.isnull().sum()

id                0
original_title    0
overview          0
dtype: int64

*Функця препроцессинга*

In [9]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    """
    На вход: датафрейм с описанием и названием фильмов,
    также пробрасываются стоп-слова и класс для лемматизации текстов
    
    На выходе: список с предобработанными текстовыми данными
    """
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]
        # Lowercase & Drop stopwords and punctuation
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        # stem_text = " ".join([stemmer.stem(word) for word in stop_free.split(' ')])
        
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    return new_text

In [10]:
nltk.download("stopwords")


text_overview = preprocessing(df_movies["overview"])
text_title = preprocessing(df_movies["original_title"])

df_movies["overview"] = text_overview
df_movies["original_title"] = text_title 

# Закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_title + text_overview

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isaev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\isaev\AppData\Local\Temp/ipykernel_3780/3915145218.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(texts))):


  0%|          | 0/45466 [00:00<?, ?it/s]

  0%|          | 0/45466 [00:00<?, ?it/s]

In [11]:
text_prepr = [i.split() for i in text_prepr]
# text_prepr_over = [i.split() for i in text_overview]

# Word2Vec

In [12]:
model = Word2Vec(text_prepr, 
                 window=10, 
                 min_count=2)

In [13]:
model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 40606


### Tockenizer & embeding

In [14]:
tokenizer = Tokenizer()

In [15]:
# Обновляем внутренний словарь на основе списка текстов.
tokenizer.fit_on_texts(text_prepr)
vocab_size = len(tokenizer.word_index) + 1

print("Total words:", vocab_size)

Total words: 71878


*Создание матрицы эмбедингов*

*Функция построения матрицы эмбедингов*

In [16]:
def embedding_matrix(tock, model):
    
    """
    На вход: словарь tokenizer.word_index 
    модель word2vec
    
    На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for word, i in tock.items():
        if word in model:
            embedding_matrix[i] = model[word]
    return embedding_matrix

In [17]:
embedding_matrix = embedding_matrix(tokenizer.word_index, model.wv)

In [18]:
df_emb = pd.concat([df_movies[['id']],
                    pd.DataFrame(embedding_matrix)], axis=1)

In [19]:
df_emb.head(5)

,id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8844,-0.269921,-0.835759,0.502752,0.750740,-0.659748,-0.556677,0.251750,0.313702,-0.333353,...,0.673201,-0.434796,0.670285,0.225371,0.337864,0.626018,0.695900,-0.729893,0.196536,0.186634
2,15602,-1.934688,-0.460044,-0.081974,0.926673,-0.828871,-0.003919,-1.382224,-0.693451,-1.050699,...,1.334867,0.148836,0.672071,0.199795,0.258788,0.720529,0.181353,0.440248,0.707728,0.066668
3,31357,-0.514307,-0.322166,0.306954,1.214350,-0.287270,-0.418287,0.319377,-0.728764,-0.720492,...,0.795687,0.213189,0.225776,0.869911,0.788290,0.841630,0.586041,0.206556,0.294887,-0.790993
4,11862,-1.226975,-0.344552,1.422143,-0.167030,-1.223665,-0.794143,-0.756765,-0.460494,-0.308596,...,1.175697,-1.312740,0.937686,-0.156272,-0.617568,1.682300,1.279618,-0.026088,-1.304477,0.577214


In [20]:
df_emb.shape

(45466, 101)